In [1]:
import tensorflow as tf
import numpy as np
from mleam.data_prep import dataset_from_json
from mleam.models import NNEmbeddingModel
import os

2024-08-21 11:46:38.129511: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-21 11:46:38.132087: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-21 11:46:38.156254: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 11:46:38.156288: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 11:46:38.156317: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
type_dict = {"Ni": 0, "Pt": 1}

dataset_train = dataset_from_json("../data/train_data.json", type_dict, batch_size=20)
dataset_val = dataset_from_json("../data/val_data.json", type_dict)

In [3]:
params = {
    ("A", "PtPt"): 0.1602,
    ("A", "NiPt"): 0.1346,
    ("A", "NiNi"): 0.0845,
    ("xi", "PtPt"): 2.1855,
    ("xi", "NiPt"): 2.3338,
    ("xi", "NiNi"): 1.405,
    ("p", "PtPt"): 13.00,
    ("p", "NiPt"): 14.838,
    ("p", "NiNi"): 11.73,
    ("q", "PtPt"): 3.13,
    ("q", "NiPt"): 3.036,
    ("q", "NiNi"): 1.93,
    ("r0", "PtPt"): 2.775,
    ("r0", "NiPt"): 2.633,
    ("r0", "NiNi"): 2.491,
    ("F_layers", "Ni"): [15, 15],
    ("F_layers", "Pt"): [15, 15],
}

for atom_types in ["PtPt", "NiPt", "NiNi"]:
    a = params[("r0", atom_types)] * np.sqrt(2)
    # Inner cutoff fourth neighbor distance (fcc)
    params[("cut_a", atom_types)] = a * np.sqrt(2)
    # Outer cutoff fifth neighbor distance (fcc)
    params[("cut_b", atom_types)] = a * np.sqrt(5 / 2)

model = NNEmbeddingModel(["Ni", "Pt"], params=params, reg=1e-5, build_forces=True)

In [4]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss={
        "energy_per_atom": tf.keras.losses.MeanSquaredError(),
        "forces": tf.keras.losses.MeanSquaredError(),
    },
)

In [5]:
run_name = "NNF/lambda1E-5/15-15/"
if not os.path.exists("./saved_models/%s" % run_name):
    os.makedirs("./saved_models/%s" % run_name)
my_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath="./saved_models/%s/model.{epoch:03d}-{val_loss:.4f}.h5" % run_name,
        save_weights_only=True,
        monitor="val_loss",
        save_best_only=True,
        mode="min",
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        mode="min",
        restore_best_weights=True,
    ),
    tf.keras.callbacks.TensorBoard(log_dir="./logs/%s/" % run_name),
]

In [6]:
model.fit(
    x=dataset_train,
    validation_data=dataset_val,
    epochs=2,
    callbacks=my_callbacks,
    initial_epoch=0,
)

Epoch 1/2
Instructions for updating:
Use fn_output_signature instead
61/61 [==============================] - 57s 514ms/step - loss: 0.2107 - energy_per_atom_loss: 0.0983 - forces_loss: 0.1121 - val_loss: 0.1039 - val_energy_per_atom_loss: 0.0260 - val_forces_loss: 0.0776
Epoch 2/2
61/61 [==============================] - 30s 487ms/step - loss: 0.1066 - energy_per_atom_loss: 0.0304 - forces_loss: 0.0759 - val_loss: 0.0939 - val_energy_per_atom_loss: 0.0203 - val_forces_loss: 0.0732


In [7]:
dataset_test = dataset_from_json("../data/test_data.json", type_dict)
model.evaluate(dataset_test)

1/1 [==============================] - 1s 903ms/step - loss: 0.1329 - energy_per_atom_loss: 0.0700 - forces_loss: 0.0626


[0.13290999829769135, 0.06997953355312347, 0.06260979175567627]

In [8]:
model.weights[0:6:2]

[<tf.Variable 'A_NiNi:0' shape=(1,) dtype=float32, numpy=array([0.05877375], dtype=float32)>,
 <tf.Variable 'A_NiPt:0' shape=(1,) dtype=float32, numpy=array([0.12281458], dtype=float32)>,
 <tf.Variable 'A_PtPt:0' shape=(1,) dtype=float32, numpy=array([0.15434523], dtype=float32)>]

In [9]:
model.weights[1:7:2]

[<tf.Variable 'p_NiNi:0' shape=(1,) dtype=float32, numpy=array([11.763502], dtype=float32)>,
 <tf.Variable 'p_NiPt:0' shape=(1,) dtype=float32, numpy=array([14.786839], dtype=float32)>,
 <tf.Variable 'p_PtPt:0' shape=(1,) dtype=float32, numpy=array([12.995765], dtype=float32)>]

In [10]:
model.weights[15:21:2]

[<tf.Variable 'xi_NiNi:0' shape=(1,) dtype=float32, numpy=array([1.4420654], dtype=float32)>,
 <tf.Variable 'xi_NiPt:0' shape=(1,) dtype=float32, numpy=array([2.303087], dtype=float32)>,
 <tf.Variable 'xi_PtPt:0' shape=(1,) dtype=float32, numpy=array([2.1893072], dtype=float32)>]

In [11]:
model.weights[16:22:2]

[<tf.Variable 'q_NiNi:0' shape=(1,) dtype=float32, numpy=array([1.9565308], dtype=float32)>,
 <tf.Variable 'q_NiPt:0' shape=(1,) dtype=float32, numpy=array([3.0835903], dtype=float32)>,
 <tf.Variable 'q_PtPt:0' shape=(1,) dtype=float32, numpy=array([3.1774182], dtype=float32)>]